In [1]:
!pip install torchinfo

In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import OxfordIIITPet
from torchvision.transforms import ToTensor, Resize
import torch.nn as N
import matplotlib.pyplot as plt
import cv2 as cv2
import itertools

In [3]:
torch.manual_seed(42)

batch_size = 32
epochs = 10
learning_rate = 9e-3
smooth = 1e-7

filters = [64, 128, 256, 512]
num_classes = 3

In [4]:
transforms = [Resize((256,256))]

In [5]:
OxfordIIITPet('../pytorch-basics/datasets', 'trainval', 'segmentation', download= True)

Dataset OxfordIIITPet
    Number of datapoints: 3680
    Root location: ../pytorch-basics/datasets

In [6]:
class OxfordPetDataset(Dataset):
    def __init__(self, split= 'trainval', transforms= []) -> None:
        self.transforms = transforms

        self.list_of_files = []
        self.input_dir = '../pytorch-basics/datasets/oxford-iiit-pet/images/'
        self.target_dir = '../pytorch-basics/datasets/oxford-iiit-pet/annotations/trimaps/'

        if(split == 'trainval'):
            file_list_path = '../pytorch-basics/datasets/oxford-iiit-pet/annotations/trainval.txt'
        else:
            file_list_path = '../pytorch-basics/datasets/oxford-iiit-pet/annotations/test.txt'

        with open(file_list_path, 'r') as f:
            for line in f.readlines():
                self.list_of_files.append(line.split(' ')[0])

    def __len__(self):
        return(len(self.list_of_files))

    def __getitem__(self, index):
        input_path = self.input_dir + self.list_of_files[index] + '.jpg'
        target_path = self.target_dir + self.list_of_files[index] + '.png'

        input = ToTensor()(cv2.imread(input_path))
        target = torch.IntTensor(cv2.imread(target_path, flags= cv2.IMREAD_UNCHANGED))

        if len(target.shape) == 2:
            target = target[None, :, :]

        for transform in self.transforms:
            input = transform(input)
            target = transform(target)

        target -= 1
        target = target.type(torch.int64)
        target = N.functional.one_hot(target, num_classes)
        target = target.squeeze().type(torch.float32).permute(2,0,1)

        return (input, target)

In [7]:
training_data = OxfordPetDataset('trainval', transforms= transforms)
test_data = OxfordPetDataset('test', transforms= transforms)

In [8]:
train_dataloader = DataLoader(training_data, batch_size= batch_size, shuffle= True, drop_last= True)
test_dataloader = DataLoader(test_data, batch_size= batch_size, shuffle= True, drop_last= True)

In [9]:
class DownwardBlock(N.Module):
    def __init__(self, in_channels, out_channels, device, stride= 1, kernel_size= 3, padding= 'same'):
        super(DownwardBlock, self).__init__()

        self.enc_block = N.Sequential(
            N.Conv2d(in_channels, out_channels, kernel_size, stride, padding, device= device, bias= False),
            N.BatchNorm2d(out_channels, device= device),
            N.ReLU(inplace= True),

            N.Conv2d(out_channels, out_channels, kernel_size, stride, padding, device= device, bias= False),
            N.BatchNorm2d(out_channels, device= device),
            N.ReLU(inplace= True)
        )

    def forward(self, input):
        output = self.enc_block(input)
        return output

In [10]:
class DownwardHalf(N.Module):
    def __init__(self, filters, device):
        super(DownwardHalf, self).__init__()
        self.down_blocks = N.ModuleList()
        self.mpool = N.MaxPool2d(kernel_size= 2, stride= 2)

        for i in range(len(filters)):
            if i == 0:
                self.down_blocks.append(DownwardBlock(3, filters[i], device= device))
            else:
                self.down_blocks.append(DownwardBlock(filters[i-1], filters[i], device= device))

    def forward(self, input):
        skip_conn = []
        for i in range(len(self.down_blocks)):
            input = self.down_blocks[i](input)
            skip_conn.append(input)
            input = self.mpool(input)

        return input, skip_conn

In [11]:
class UpwardBlock(N.Module):
    def __init__(self, in_channels, out_channels, device, stride= 1, kernel_size= 3, padding='same'):
        super(UpwardBlock, self).__init__()

        self.tconv = N.ConvTranspose2d(in_channels, out_channels, kernel_size= 2, stride= 2, device= device)
        self.dec_block = N.Sequential(
            N.Conv2d(in_channels, out_channels, kernel_size, stride, padding, device= device, bias= False),
            N.BatchNorm2d(out_channels, device= device),
            N.ReLU(inplace= True),

            N.Conv2d(out_channels, out_channels, kernel_size, stride, padding, device= device, bias= False),
            N.BatchNorm2d(out_channels, device= device),
            N.ReLU(inplace= True)
        )

    def forward(self, input, skip_conn):
        tconv_output = self.tconv(input)
        concat_output = torch.cat([tconv_output, skip_conn], dim= 1)
        output = self.dec_block(concat_output)

        return output

In [12]:
class UpwardHalf(N.Module):
    def __init__(self, filters, device):
        super(UpwardHalf, self).__init__()
        self.up_blocks = N.ModuleList()
        n = len(filters)

        for i in range(n):
          self.up_blocks.append(UpwardBlock(filters[n-i-1] * 2, filters[n-i-1], device))

    def forward(self, input, skip_conn):
        n = len(self.up_blocks)
        for i in range(n):
            input = self.up_blocks[i](input, skip_conn[n-i-1])

        return input

In [13]:
class UNet(N.Module):
    def __init__(self, filters, device):
        super(UNet, self).__init__()
        self.down_half = DownwardHalf(filters, device)
        self.up_half = UpwardHalf(filters, device)
        self.final_conv = N.Conv2d(filters[0], num_classes, 3, 1, 'same', device= device)
        self.bridge_block = N.Sequential(
            N.Conv2d(filters[-1], filters[-1] * 2, 3, 1, 'same', device= device, bias= False),
            N.BatchNorm2d(filters[-1] * 2, device= device),
            N.ReLU(inplace= True),

            N.Conv2d(filters[-1] * 2, filters[-1] * 2, 3, 1, 'same', device= device, bias= False),
            N.BatchNorm2d(filters[-1] * 2, device= device),
            N.ReLU(inplace= True)
        )

    def forward(self, input):
        down_half_output, skip_conn = self.down_half(input)
        bridge_block_output = self.bridge_block(down_half_output)
        up_half_output = self.up_half(bridge_block_output, skip_conn)
        output = self.final_conv(up_half_output)

        return output

In [14]:
from torchinfo import summary


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(filters, device)

summary(model, input_size= (batch_size, 3, 256, 256), col_names=['input_size', 'output_size', 'num_params', 'trainable'],

        row_settings=['var_names'], verbose=0)

Layer (type (var_name))                       Input Shape               Output Shape              Param #                   Trainable
UNet (UNet)                                   [32, 3, 256, 256]         [32, 3, 256, 256]         --                        True
├─DownwardHalf (down_half)                    [32, 3, 256, 256]         [32, 512, 16, 16]         --                        True
│    └─ModuleList (down_blocks)               --                        --                        (recursive)               True
│    │    └─DownwardBlock (0)                 [32, 3, 256, 256]         [32, 64, 256, 256]        38,848                    True
│    └─MaxPool2d (mpool)                      [32, 64, 256, 256]        [32, 64, 128, 128]        --                        --
│    └─ModuleList (down_blocks)               --                        --                        (recursive)               True
│    │    └─DownwardBlock (1)                 [32, 64, 128, 128]        [32, 128, 128, 128]   

In [15]:
def dice_score(output, target):
    output_flattened = output.flatten()
    target_flattened = target.flatten()
    intersection = torch.sum(output_flattened * target_flattened)

    dice_score = (2. * intersection + smooth)/(torch.sum(output_flattened) + torch.sum(target_flattened) + smooth)
    return dice_score

def dice_score_multiclass(output, target, num_classes):
    output = torch.argmax(output, dim= 1).type(torch.int64)
    output = N.functional.one_hot(output, num_classes).permute(0,3,1,2)
    target = target.type(torch.int64)

    dice= 0
    for idx in range(num_classes):
        dice += dice_score(output[:,idx,:,:], target[:,idx,:,:])

    return dice/num_classes

In [16]:
loss_fn = N.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

In [ ]:
for _ in range(epochs):
    train_loss = 0
    print(f"\nStarting epoch {_ + 1}\n~~~~~~~~~~~~~~")

    for index, (input, target) in enumerate(train_dataloader):
        input, target = input.cuda(), target.cuda()
        model.train()
        output = model(input)
        output = N.Softmax(dim= 1)(output)

        loss = loss_fn(output, target)
        train_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(index % 50 == 0):
            print(f"Batch: {index} Loss:{loss:0.9f}")

    train_loss /= len(train_dataloader)
    test_loss, test_dice = 0, 0
    model.eval()

    with torch.inference_mode():
        for test_input, test_target in test_dataloader:
            test_input, test_target = test_input.cuda(), test_target.cuda()

            test_output = model(test_input)
            test_output = N.Softmax(dim= 1)(test_output)
            test_loss += loss_fn(test_output, test_target)
            test_dice += dice_score_multiclass(test_target, test_output, num_classes)

        test_loss /= len(test_dataloader)
        test_dice /= len(test_dataloader)

    print(f"Train loss: {train_loss:0.5f} Test loss: {test_loss:0.5f} Test dice score: {test_dice:0.5f}")


Starting epoch 1
~~~~~~~~~~~~~~
Batch: 0 Loss:1.105314374
Batch: 50 Loss:0.822457671
Batch: 100 Loss:0.808479190
Train loss: 0.84978 Test loss: 0.82083 Test dice score: 0.26850

Starting epoch 2
~~~~~~~~~~~~~~
Batch: 0 Loss:0.832635283
Batch: 50 Loss:0.782255232
Batch: 100 Loss:0.795452833
Train loss: 0.80102 Test loss: 0.83191 Test dice score: 0.28930

Starting epoch 3
~~~~~~~~~~~~~~
Batch: 0 Loss:0.794442296
Batch: 50 Loss:0.808420539


In [ ]:
sample_index =  torch.randint(0, len(test_dataloader), size= (1,))
sample_input, sample_target = next(itertools.islice(test_dataloader, sample_index, None))

In [ ]:
sample_output = model(sample_input.cuda())

In [ ]:
sample_output = torch.argmax(N.Softmax(dim= 1)(sample_output), dim= 1)

In [ ]:
idx = torch.randint(0, 32, size= (1,))

plt.imshow(sample_output[idx].squeeze().cpu())
plt.show()
plt.imshow(sample_target[idx].squeeze().permute(1,2,0))
plt.show()
plt.imshow(sample_input[idx].squeeze().permute(1,2,0))

In [ ]:
torch.save(model, '../')